In [33]:
import numpy as np
import pandas as pd
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, GlobalMaxPool1D, Conv1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
df = pd.read_csv("news.csv")
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
labels = df['label']
labels.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [7]:
text = df['text']
text

0       Daniel Greenfield, a Shillman Journalism Fello...
1       Google Pinterest Digg Linkedin Reddit Stumbleu...
2       U.S. Secretary of State John F. Kerry said Mon...
3       — Kaydee King (@KaydeeKing) November 9, 2016 T...
4       It's primary day in New York and front-runners...
                              ...                        
6330    The State Department told the Republican Natio...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332     Anti-Trump Protesters Are Tools of the Oligar...
6333    ADDIS ABABA, Ethiopia —President Obama convene...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: text, Length: 6335, dtype: object

In [6]:
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels

array([0, 0, 1, ..., 0, 1, 1])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
  text, labels, test_size=0.2, random_state=42)

In [9]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

In [10]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [11]:
vocab_size = len(tokenizer.word_index) + 1

In [17]:
# accounting for text of different length
max_len = 500

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [18]:
X_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [19]:
embedding_dim = 50

In [30]:
model = Sequential([
  Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
  GlobalMaxPool1D(),
  Flatten(),
  Dense(10, activation="relu"),
  Dense(1, 'sigmoid')
])

model.compile(
  loss="binary_crossentropy",
  metrics=['accuracy'],
  optimizer="adam"
)

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 50)           4395150   
                                                                 
 global_max_pooling1d (Globa  (None, 50)               0         
 lMaxPooling1D)                                                  
                                                                 
 flatten_1 (Flatten)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 10)                510       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 4,395,671
Trainable params: 4,395,671
Non-trainable params: 0
____________________________________________

In [32]:
history = model.fit(
  X_train, y_train,
  epochs=20,
  validation_data=(X_test, y_test)
)

Epoch 1/20
159/159 [==============================] - 12s 71ms/step - loss: 0.6542 - accuracy: 0.7701 - val_loss: 0.5584 - val_accuracy: 0.8674
Epoch 2/20
159/159 [==============================] - 10s 63ms/step - loss: 0.3934 - accuracy: 0.8887 - val_loss: 0.2710 - val_accuracy: 0.9053
Epoch 3/20
159/159 [==============================] - 10s 65ms/step - loss: 0.2166 - accuracy: 0.9240 - val_loss: 0.2026 - val_accuracy: 0.9179
Epoch 4/20
159/159 [==============================] - 13s 80ms/step - loss: 0.1489 - accuracy: 0.9509 - val_loss: 0.1797 - val_accuracy: 0.9211
Epoch 5/20
159/159 [==============================] - 14s 86ms/step - loss: 0.1051 - accuracy: 0.9716 - val_loss: 0.1736 - val_accuracy: 0.9250
Epoch 6/20
159/159 [==============================] - 11s 66ms/step - loss: 0.0750 - accuracy: 0.9807 - val_loss: 0.1677 - val_accuracy: 0.9227
Epoch 7/20
159/159 [==============================] - 9s 60ms/step - loss: 0.0529 - accuracy: 0.9901 - val_loss: 0.1678 - val_accuracy: 